# Extract and analyze a movie

미디어 & 엔터테인먼트, 광고 및 스포츠와 같은 산업은 TV 쇼, 영화, 뉴스, 스포츠 이벤트, 다큐멘터리 등을 포함한 방대한 전문 제작 비디오 인벤토리를 관리합니다. 이러한 유형의 비디오 콘텐츠에서 효과적으로 인사이트를 추출하기 위해, 사용자들은 비디오 요약, 장면 수준 분석, 광고 타겟팅을 위한 IAB 분류, 화자 식별과 같은 정보가 필요합니다.

> [IAB 카테고리](https://smartclip.tv/adtech-glossary/iab-categories/)는 Interactive Advertising Bureau (IAB)에서 개발한 웹 콘텐츠에 대한 표준 분류입니다. 이러한 카테고리들은 광고주들을 산업과 세그먼트로 분류하는 데 사용됩니다.

이 실습에서는 BDA Video를 사용하여 오픈소스 샘플 영화인 Meridian을 추출하고 분석할 것입니다. 이 과정을 통해 프로세스를 살펴보고 생성된 출력물들을 탐색할 것입니다.

## Prerequisites

### 이전 노트북에서 저장된 값 불러오기

이 노트북을 실행하려면 이전 노트북인 [00_prerequisites.ipynb](./00-prequisites.ipynb)를 실행했어야 합니다. 해당 노트북에서 패키지 종속성을 설치하고 SageMaker 환경에서 일부 정보를 수집했습니다.

In [ ]:
store -r

In [ ]:
data_bucket = session["bucket"]
region = sagemaker_resources["region"]
data_prefix = "bda/video"
output_prefix = "bda/video/ouput"

In [ ]:
import boto3
import json
import uuid
import utils

bda_client = boto3.client('bedrock-data-automation')
bda_runtime_client = boto3.client('bedrock-data-automation-runtime')
s3_client = boto3.client('s3')

#access account id
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]

default_profile_arn = f"arn:aws:bedrock:{region}:{account_id}:data-automation-profile/us.data-automation-v1"

## BDA 프로젝트 생성 및 비디오를 위한 표준 출력 구성 설정

BDA 작업을 시작하려면 표준 및 사용자 지정 출력 구성을 모두 관리하는 BDA 프로젝트가 필요합니다. 이 프로젝트는 재사용이 가능하며, 동일한 설정을 공유하는 여러 비디오를 처리하는 데 동일한 구성을 적용할 수 있습니다.

아래 코드 스니펫에서는 비디오 모달리티를 위한 표준 출력 구성으로 BDA 프로젝트를 생성합니다. 이러한 구성은 필요한 특정 정보만 추출하도록 조정할 수 있습니다. 이 실습에서는 다음과 같은 비디오 출력을 활성화할 것입니다:
- 전체 비디오 요약
- 장면 요약
- 장면 수준의 IAB 카테고리
- 전체 오디오 트랜스크립트
- 경계 상자가 있는 비디오 내 텍스트

BDA 프로젝트 생성을 위한 전체 API 참조는 이 [문서](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-data-automation/client/create_data_automation_project.html)를 참조하세요.

In [ ]:
response = bda_client.create_data_automation_project(
    projectName=f'bda-workshop-video-project-{str(uuid.uuid4())[0:4]}',
    projectDescription='BDA workshop video sample project',
    projectStage='DEVELOPMENT',
    standardOutputConfiguration={
        'video': {
            'extraction': {
                'category': {
                    'state': 'ENABLED',
                    'types': ['TEXT_DETECTION','TRANSCRIPT'],
                },
                'boundingBox': {
                    'state': 'ENABLED',
                }
            },
            'generativeField': {
                'state': 'ENABLED',
                'types': ['VIDEO_SUMMARY','CHAPTER_SUMMARY','IAB'],
            }
        }
    }
)

create_data_automation_project API는 프로젝트 ARN을 반환할 것이며, 이를 비디오 분석 작업을 호출하는 데 사용할 것입니다.

In [ ]:
video_project_arn = response.get("projectArn")
print("BDA video project ARN:", video_project_arn)

## 영화를 추출하고 분석하기 위한 비동기 BDA 작업 시작하기
이 섹션에서는 오픈소스 영화인 Meridian을 사용하여 BDA 프로젝트에서 정의된 구성을 적용하여 BDA로 추출하고 분석할 것입니다. 그런 다음 출력을 검토하여 BDA가 비디오 추출 및 분석을 수행하는 방법에 대해 더 깊이 이해할 것입니다.

### 샘플 비디오 준비

In [ ]:
sample_video_movie = './NetflixMeridian.mp4'
!curl "https://ws-assets-prod-iad-r-pdx-f3b3f9f1a7d6a3d0.s3.us-west-2.amazonaws.com/7db2455e-0fa6-4f6d-9973-84daccd6421f/Netflix_Open_Content_Meridian.mp4" --output NetflixMeridian.mp4


비디오를 표시해 보겠습니다. [Meridian](https://en.wikipedia.org/wiki/Meridian_(film))은 Netflix의 테스트 영화로, BDA가 비디오 추출과 함께 어떻게 작동하는지 보여주기 위해 사용합니다. 보시다 시피 여러 장면으로 구성된 클래식 스타일의 영화입니다.

In [ ]:
from IPython.display import Video
Video(sample_video_movie, width=800)

비디오를 BDA로 분석하기 위해서는 BDA가 접근할 수 있는 S3 버킷에 업로드해야 합니다.

In [ ]:
s3_key = f'{data_prefix}/{sample_video_movie.split("/")[-1]}'
s3_client.upload_file(sample_video_movie, data_bucket, s3_key)

### BDA 작업 시작
이제 업로드된 비디오를 처리하기 위해 BDA API를 호출할 것입니다. 실습 초반에 생성한 BDA 프로젝트 ARN을 제공하고 BDA가 출력 결과를 저장할 S3 위치를 지정해야 합니다.

BDA 비동기 작업 호출을 위한 전체 API 참조는 이 [문서](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-data-automation-runtime/client/invoke_data_automation_async.html)를 참조하세요.

In [ ]:
response = bda_runtime_client.invoke_data_automation_async(
    inputConfiguration={
        's3Uri': f's3://{data_bucket}/{s3_key}'
    },
    outputConfiguration={
        's3Uri': f's3://{data_bucket}/{output_prefix}'
    },
    dataAutomationConfiguration={
        'dataAutomationProjectArn': video_project_arn,
        'stage': 'DEVELOPMENT'
    },
    notificationConfiguration={
        'eventBridgeConfiguration': {
            'eventBridgeEnabled': False
        }
    },
    dataAutomationProfileArn=default_profile_arn
)

`invoke_data_automation_async` API는 비동기식입니다. 이는 호출 작업 식별자인 `invocationArn`을 반환합니다. 그런 다음 다른 API인 `get_data_automation_status`를 사용하여 작업이 완료될 때까지 작업의 상태를 모니터링할 수 있습니다.

> 프로덕션 워크로드에서는 이벤트 기반 패턴이 권장됩니다. 작업이 완료되면 BDA가 다음 단계를 트리거하도록 합니다. 이는 Lambda 함수와 같은 구독된 AWS 서비스에 알림을 보내는 invoke 작업에서 notificationConfiguration을 구성하여 달성할 수 있습니다. 또는 BDA가 결과를 저장할 버킷에 S3 트리거를 설정할 수 있습니다.

In [ ]:
invocation_arn = response.get("invocationArn")
print("BDA task started:", invocation_arn)

이 실습에서는 작업이 완료될 때까지 5초마다 `get_data_automation_status` API를 호출하여 작업을 모니터링하기 위해 아래 루프를 사용할 것입니다.

이 비디오는 약 5-10분이 소요됩니다.

In [ ]:
import time
from IPython.display import clear_output
from datetime import datetime

status, status_response = None, None
while status not in ["Success","ServiceError","ClientError"]:
    status_response = bda_runtime_client.get_data_automation_status(
        invocationArn=invocation_arn
    )
    status = status_response.get("status")
    clear_output(wait=True)
    print(f"{datetime.now().strftime('%H:%M:%S')} : BDA video task: {status}")
    time.sleep(5)

output_config = status_response.get("outputConfiguration",{}).get("s3Uri")
print("Ouput configureation file:", output_config)

## BDA 분석 결과 접근하기
`get_data_automation_status` API는 BDA가 추출 결과를 출력하는 S3 위치를 제공하는 결과 구성이 포함된 S3 URI를 반환합니다. 그런 다음 이 파일을 구문 분석하여 결과 경로를 검색할 것입니다.

In [ ]:
config_data = utils.read_json_on_s3(output_config,s3_client)
print(json.dumps(config_data, indent=4))

위에서 보듯이, BDA 출력 구성 파일에는 작업 ID, 상태, 모달리티 및 실제 결과 JSON의 S3 위치를 포함하여 BDA 결과에 대한 메타데이터가 포함되어 있습니다. 이제 이 결과 파일을 다운로드하여 출력을 확인할 것입니다.

In [ ]:
from IPython.display import JSON

result_uri = config_data["output_metadata"][0]["segment_metadata"][0]["standard_output_path"]
result_data = utils.read_json_on_s3(result_uri,s3_client)

JSON(result_data)

## 결과 검토
BDA 비디오 분석 결과는 비디오 및 장면 수준으로 구성된 상세한 정보 분석을 포함합니다.
> 비디오 장면은 비디오 내에서 일관된 행동이나 내러티브 단위를 형성하는 일련의 샷들입니다.

### 전체 비디오 요약

비디오 수준의 요약을 살펴보겠습니다 - 이는 비디오 전반에 걸쳐 제시된 주요 주제, 이벤트 및 정보를 간단한 요약으로 압축한 것입니다.

In [ ]:
print(result_data["video"]["summary"])

### 전체 비디오 트랜스크립션
비디오 수준에서는 화자가 식별된 비디오의 오디오를 기반으로 한 전체 트랜스크립트도 받습니다.

In [ ]:
print(result_data["video"]["transcript"]["representation"]["text"])

### 장면 정의, 요약 및 IAB 카테고리
BDA는 또한 프로젝트 구성에 지정된 대로 장면 수준의 요약을 생성합니다. 추가적으로 각 장면의 시작 및 종료 시간과 함께 장면 콘텐츠를 기반으로 분류된 [IAB](https://en.wikipedia.org/wiki/Interactive_Advertising_Bureau) 카테고리를 포함한 더 많은 메타데이터를 얻을 수 있습니다.

In [ ]:
for chapter in result_data["chapters"]:
    iabs = []
    if chapter.get("iab_categories"):
        for iab in chapter["iab_categories"]:
            iabs.append(iab["category"])
        
    print(f'[{chapter["start_timecode_smpte"]} - {chapter["end_timecode_smpte"]}] {", ".join(iabs)}')
    print(chapter["summary"])
    print()

### 상세 오디오 트랜스크립트
장면 수준에서도 상세 트랜스크립트를 사용할 수 있습니다. 각 장면 아래에서 타임스탬프가 포함된 `audio_segments`라는 목록을 찾을 수 있습니다. 이는 상세한 트랜스크립트 정보가 필요한 추가적인 다운스트림 분석을 지원할 수 있습니다.

In [ ]:
for chapter in result_data["chapters"]:
    for trans in chapter["audio_segments"]:
        print(f'[{trans["start_timestamp_millis"]/1000} - {trans["end_timestamp_millis"]/1000}] {trans["text"]}')

### 프레임 수준의 텍스트 추출과 경계 상자 및 신뢰도 점수
텍스트 추출은 경계 상자 및 신뢰도 점수와 함께 프레임 수준에서 사용할 수 있습니다. 출력 JSON 구조에서 프레임은 캡처된 타임스탬프와 함께 각 장면 아래에 구성됩니다. 주어진 프레임에서 텍스트가 감지되면 프레임 수준에서 text_words와 text_lines를 찾을 수 있습니다.

감지된 텍스트가 있는 주어진 장면에 대한 프레임을 경계 상자와 함께 표시해 보겠습니다.

In [ ]:
# plot all frames with boundingbox in the given scene
chapter_index = 1 

width = result_data["metadata"]["frame_width"]
height = result_data["metadata"]["frame_height"]

from moviepy.video.io.VideoFileClip import VideoFileClip
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

with VideoFileClip(sample_video_movie) as video_clip:
    for chapter in result_data["chapters"]:
        if chapter["chapter_index"] == chapter_index:
            for frame in chapter["frames"]:
                bboxes = []
                if frame.get("text_lines"):
                    for tl in frame["text_lines"]:
                        for l in tl["locations"]:
                            bbox = l["bounding_box"]
                            if bbox:
                                bboxes.append((
                                                width*bbox["left"], 
                                                height*bbox["top"], 
                                                width * (bbox["width"]+bbox["left"]), 
                                                height * (bbox["height"] + bbox["top"])
                                            ))
                if bboxes:
                    timestamp = frame["timestamp_millis"]/1000
                    frame = video_clip.get_frame(timestamp)  
                    frame_image = Image.fromarray(frame)
                    draw = ImageDraw.Draw(frame_image)
                    for box in bboxes:
                        draw.rectangle(box, outline="red", width=2)

                    plt.figure(figsize=(10, 6))
                    plt.imshow(frame_image)
                    plt.title(f"Frame at {timestamp} seconds with Bounding Boxes")
                    plt.axis("off")
                    plt.show()

## 요약
BDA 비디오는 구성에 의해 관리되는 매우 상세한 메타데이터를 반환합니다. 이 실습에서는 영화를 예시로 사용하여 미디어 비디오 분석에 필요한 표준 출력을 활성화했습니다. 출력 JSON을 탐색하여 더 많은 세부 정보를 발견할 수 있습니다. 이 실습에서는 콘텐츠 규제/제한 감지 및 분석을 다루지 않습니다. 이를 위해서는 BDA 비디오가 제공하는 조정 분석을 더 잘 이해하기 위해 소셜 미디어 스타일의 비디오를 예시로 사용하는 다음 실습을 참조할 수 있습니다.

## 정리

BDA 프로젝트, 블루프린트, 이미지 및 S3의 결과를 삭제합니다.

In [ ]:
# delete BDA project
response = bda_client.delete_data_automation_project(
    projectArn=video_project_arn
)
response

In [ ]:
# delete uploaded image from S3
s3_client.delete_object(Bucket=data_bucket, Key=s3_key)

## 워크샵의 다음 섹션으로 계속하기

워크샵의 다음 섹션에서는 BDA에서 생성된 메타데이터를 사용하여 사용 사례를 해결할 것입니다.

다음 노트북으로 이동하세요 [문맥 기반 광고 오버레이]([02-contextual-ad-overlay-kr.ipynb])